In [ ]:
# sunshine = 2,0
c1=0
c2=3
c3=4
LNB1=LONB[:,c1][LONB[:,c1]!=0]
LTB1=LATB[:,c1][LATB[:,c1]!=0]
CDB1=CODB[:,c1][CODB[:,c1]!=0]
ATB1=ALTB[:,c1][ALTB[:,c1]!=0]
print [LNB1.size,LTB1.size,CDB1.size,ATB1.size]
LNB2=LONB[:,c2][LONB[:,c2]!=0]
LTB2=LATB[:,c2][LATB[:,c2]!=0]
CDB2=CODB[:,c2][CODB[:,c2]!=0]
ATB2=ALTB[:,c2][ALTB[:,c2]!=0]
print [LNB2.size,LTB2.size,CDB2.size,ATB2.size]
# precip = 0,1
c1=0
c2=1
LNL1=LONL[:,c1][LONL[:,c1]!=0]
LTL1=LATL[:,c1][LATL[:,c1]!=0]
CDL1=CODL[:,c1][CODL[:,c1]!=0]
ATL1=ALTL[:,c1][ALTL[:,c1]!=0]
print [LNL1.size,LTL1.size,CDL1.size,ATL1.size]
LNL2=LONL[:,c2][LONL[:,c2]!=0]
LTL2=LATL[:,c2][LATL[:,c2]!=0]
CDL2=CODL[:,c2][CODL[:,c2]!=0]
ATL2=ALTL[:,c2][ALTL[:,c2]!=0]
print [LNL2.size,LTL2.size,CDL2.size,ATL2.size]
# rainy days: 2,0
c1=2
c2=0
LND1=LOND[:,c1][LOND[:,c1]!=0]
LTD1=LATD[:,c1][LATD[:,c1]!=0]
CDD1=CODD[:,c1][CODD[:,c1]!=0]
ATD1=ALTD[:,c1][ALTD[:,c1]!=0]
print [LND1.size,LTD1.size,CDD1.size,ATD1.size]
LND2=LOND[:,c2][LOND[:,c2]!=0]
LTD2=LATD[:,c2][LATD[:,c2]!=0]
CDD2=CODD[:,c2][CODD[:,c2]!=0]
ATD2=ALTD[:,c2][ALTD[:,c2]!=0]
print [LND2.size,LTD2.size,CDD2.size,ATD2.size]
# humidity 1,3 (might be the opposite?)
c1=1
c2=3
LNH1=LONH[:,c1][LONH[:,c1]!=0]
LTH1=LATH[:,c1][LATH[:,c1]!=0]
CDH1=CODH[:,c1][CODH[:,c1]!=0]
ATH1=ALTH[:,c1][ALTH[:,c1]!=0]
print [LNH1.size,LTH1.size,CDH1.size,ATH1.size]
LNH2=LONH[:,c2][LONH[:,c2]!=0]
LTH2=LATH[:,c2][LATH[:,c2]!=0]
CDH2=CODH[:,c2][CODH[:,c2]!=0]
ATH2=ALTH[:,c2][ALTH[:,c2]!=0]
print [LNH2.size,LTH2.size,CDH2.size,ATH2.size]

In [ ]:
def mcsel(big,small):
    MC=np.zeros(big)
    for i in range(big):
        MC[i]=np.random.random()<small*1./big
    return MC.astype(bool)

In [ ]:
# stackear D y L y hacer el MC sobre esos
MCclp=np.empty((0))
MClnp=np.empty((0))
MCltp=np.empty((0))
MCatp=np.empty((0))
iterations=10000
for i in range(iterations):
    DMC=mcsel(LND1.size,LNB1.size)
    HMC=mcsel(LNH1.size,LNB1.size)
    LMC=mcsel(LNL1.size,LNB1.size)
    LNMC=np.hstack((LNB1,LND1[DMC],LNH1[HMC],LNL1[LMC]))
    LTMC=np.hstack((LTB1,LTD1[DMC],LTH1[HMC],LTL1[LMC]))
    ATMC=np.hstack((ATB1,ATD1[DMC],ATH1[HMC],ATL1[LMC]))
    CDMC=np.hstack((CDB1,CDD1[DMC],CDH1[HMC],CDL1[LMC]))
    MCdata=np.vstack((LNMC,LTMC,ATMC)).T
    kcomp=2
    kik=0
    kikp=1
    gmm = GMM(kcomp, covariance_type='full', random_state=0)
    gmm.fit(MCdata)
    cluster_label = gmm.predict(MCdata)
    MCcl=np.append(MCcl,CDMC[cluster_label==kik])
    MCln=np.append(MCln,LNMC[cluster_label==kik])
    MClt=np.append(MClt,LTMC[cluster_label==kik])    
    MCat=np.append(MCat,ATMC[cluster_label==kik])
    MCclp=np.append(MCclp,CDMC[cluster_label==kikp])
    MClnp=np.append(MClnp,LNMC[cluster_label==kikp])
    MCltp=np.append(MCltp,LTMC[cluster_label==kikp])    
    MCatp=np.append(MCatp,ATMC[cluster_label==kikp])
   
np.savetxt('mcresults.txt',np.c_[MCcl,MCln,MClt,MCat])
np.savetxt('mcresultsp.txt',np.c_[MCclp,MClnp,MCltp,MCatp])

In [ ]:
%%bash

cat mcresults.txt | sort -n | uniq -c | sort -nr > mcrescount.txt
cat mcresultsp.txt | sort -n | uniq -c | sort -nr > mcrescountp.txt

In [ ]:
A=np.loadtxt('mcrescount.txt')
FILTER=A[:,0]>800
plt.scatter(-A[:, 2][FILTER], -A[:, 3][FILTER], c=A[:,0][FILTER])
plt.colorbar()
plt.xlim(-85,-65)
plt.ylim(-5,14)

In [ ]:
A=np.loadtxt('mcrescountp.txt')
FILTER=A[:,0]>1000
plt.scatter(-A[:, 2][FILTER], -A[:, 3][FILTER], c=A[:,0][FILTER])
plt.colorbar()
plt.xlim(-85,-65)
plt.ylim(-5,14)

In [ ]:
def genprob(paux3,paux4):
    p3=1./4
    p4=1./16
    p33=p3*paux3
    p34=p4*paux4
    lott=np.random.random(1)
    if(lott<(p33+p34)):
        if(lott<p33):
            return 3
        elif(p33<=lott<p33+p34):
            return 4
    else:
         return 0    

In [ ]:
def parseprob(i,j):
    if((i==4.)&(j==0.)):
        return genprob(0.,1.)
    if((i==3.)&(j==0.)):
        return genprob(1.,0.)
    if((i==3.)&(j==1.)):
        return genprob(4./5,1./5)
    if((i==2.)&(j==1.)):
        return genprob(4./10,0.)
    if((i==2.)&(j==2.)):
        return genprob(4./11,1./11)
    if((i==1.)&(j==2.)):
        return genprob(4./14,0.)
    if((i==1.)&(j==3.)):
        return genprob(4./15,1./15)